Turning mnemonic seed to master private key

1. Perform a key streching function with 2048 rounds of hashing using HMAC. i.e. PBKDF2 with SHA256. PBKDF2 has 5 input parameters: a pseudorandom function (1), password (2), a salt (3), a number of iterations (4), and a key length in bytes (5).

In our case, (1) the Password: 12 mnemonic code words, (2)salt is the passhrase: "mnemonic"+ an optional phrase (3) number of iterations is 2048 , and (4) key length is 512 bits (64 bytes)



In [3]:
import hashlib

passphrase = "SuperDuperSecret"
passphrase = "mnemonic"+passphrase
mnemonic = "army van defense carry jealous true garbage claim echo media make crunch"

passphrase_bytes = bytes(passphrase, 'utf-8')
mnemonic_bytes = bytes(mnemonic, 'utf-8')

itr = 2048
key_length = 512 #bits

#hashlib.pbkdf2_hmac(hash_name, password, salt, iterations, dklen=None)
stretch_seed = hashlib.pbkdf2_hmac("sha512", mnemonic_bytes, passphrase_bytes, itr, key_length//8)

print(stretch_seed)


b';]\xf1m\xf2\x15q\x04\xcf\xdd"\x83\x01b\xa5\xe1p\xc0\x16\x16S\xe3\xaf\xe6\xc8\x8d\xef\xee\xfb\x08\x18\xc7\x93\xdb\xb2\x8a\xb3\xab\t\x18\x97\xd0qXa\xdc\x8a\x185\x8f\x80\xb7\x9dI\xac\xf6AB\xaeW\x03}\x1dT'


2. Generating Master Key. 

(1) Compute HMAC-SHA512 of seed with key being "Bitcoin seed"

(2) Serialize following the specification in bip0032 https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki#Serialization_format

i. 4 byte: version bytes (mainnet: 0x0488B21E public, 0x0488ADE4 private; testnet: 0x043587CF public, 0x04358394 private)

ii. 1 byte: depth: 0x00 for master nodes, 0x01 for level-1 derived keys, ....

iii. 4 bytes: the fingerprint of the parent's key (0x00000000 if master key)

iv. 4 bytes: child number. This is ser32(i) for i in xi = xpar/i, with xi the key being serialized. (0x00000000 if master key)

v. 32 bytes: the chain code

vi. 33 bytes: the public key or private key data (serP(K) for public keys, 0x00 || ser256(k) for private keys)


(3) Perform a SHA-256 checksum achieved by Double hash using sha256 the result from (2), then selecting the first 32 bits (4 bytes)
(4) Returning th base58

In [11]:
import hmac
import base58

#computing HMAC-SHA 512 
# hmac.new(key, msg=None, digestmod='')¶
key = bytes("Bitcoin seed", 'utf-8')
seed = hmac.new(key, stretch_seed, digestmod=hashlib.sha512).digest()

master_key = seed[:32] #master key
chain_code = seed[32:]

#serialization 
#assumption: using a testnet
xprv = b"\x04\x88\xAD\xE4" 
xprv += b"\x00"* 9 # depth, fingerprint of parents key, and child number
xprv += chain_code
xprv += b"\x00" + master_key

#Double hashing and converting to base58
hashed_xprv = hashlib.sha256(xprv).digest()
hashed_xprv = hashlib.sha256(hashed_xprv).digest()

#selecting first 32 bits and appending it to the serialized result
xprv += hashed_xprv[:4]

root_key = base58.b58encode(xprv)

print(root_key)

b'xprv9s21ZrQH143K2Fi7kfpyykRksvVqvQsS6gfKA39LYFzUzPuJnuAiwtqWm8UZH8UcPv9neGxEUbnQVwEvsnk8cXjncGwo7LRtaywUgKQ6LLy'
